In [1]:
import os
import utils
import argparse
import numpy as np
import pandas as pd
import compress_pickle as cpickle
     

from datetime import date
from datetime import datetime

import sqlalchemy as sql

import warnings
warnings.filterwarnings('ignore')


In [2]:
infolder1 = 'ActualizacionClasificacionPrestaciones/'
outfolder1= 'ActualizacionClasificacionPrestaciones/'

### PREPARACION DATAFRAME ACTUALIZADO

In [3]:
#Me conecto al SQL
cnxn = utils.CONNECT_SQL(server='des02')
# Leo excel con pretaciones 
file1 = utils.GET_LAST_FILE(infolder1,'Prestaciones')
data1 = pd.read_excel(file1, sheet_name='TABLA')

# Exporto a un pickle comprimido
filename1 = outfolder1 + 'ClasificacionPrestaciones' + utils.GET_NOW_STRING() + '.lzma'
utils.SAVE_CPICKLE(data1, filename1)
# Leo la consulta que obtiene la fechade las prestaciones
file2 = utils.GET_LAST_FILE(infolder1,'FechaPrestacion')
data2 = utils.GET_SQL_DATA(file2,cnxn)

# Exporto a un pickle comprimido nuevamente
filename2 = outfolder1 + 'FechaPrestaciones' + utils.GET_NOW_STRING() + '.lzma'
utils.SAVE_CPICKLE(data2, filename2)

In [23]:
data2

,CodigoVC,FechaPrestacion
0,1001001,2023-12-04 10:30:40.743
1,1001003,2023-12-04 10:30:40.743
2,1001005,2023-12-04 10:30:40.743
3,1001007,2023-12-04 10:30:40.743
4,1001008,2023-12-04 10:30:40.743
...,...,...
3364,994016137,2023-12-04 10:30:40.743
3365,994016147,2023-12-04 10:30:40.743
3366,994016149,2023-12-04 10:30:40.743
3367,994016454,2023-12-04 10:30:40.743


In [39]:
#Filtro para tener prestaciones ya clasificadas
clasificados1 = data1[data1['Estado'] == 1]
clasificados1.columns
#Escojo los atributos que calzan con SQL
clasificados2= clasificados1[['CodigoVC','GlosaCorregida','GlosaProcesoVC','GlosaImed','ClasificacionGrupo','ClasificacionSubgrupo','ClasificacionApertura']]
#Cambio el formato de CodigoVC a string como en el SQL
clasificados2.loc[:, 'CodigoVC'] = clasificados2.CodigoVC.astype(str)
#Uno la query de Sql con clasificados via merge join
clasificados3 = pd.merge(clasificados2, data2, left_on='CodigoVC', right_on='CodigoVC', how='left')
clasificados3['Vigencia'] = 1
clasificados3.FechaPrestacion = [x.strftime('%Y-%m-%dT%H:%M:%S') for x in clasificados3.FechaPrestacion]


In [40]:
clasificados3['FechaPrestacion']

0      2023-12-04T10:30:40
1      2023-12-04T10:30:40
2      2023-12-04T10:30:40
3      2023-12-04T10:30:40
4      2023-12-04T10:30:40
              ...         
981    2023-12-04T10:30:40
982    2023-12-04T10:30:40
983    2023-12-04T10:30:40
984    2023-12-04T10:30:40
985    2023-12-04T10:30:40
Name: FechaPrestacion, Length: 986, dtype: object

### GENERACION DE SCRIPT SQL

In [43]:
def prepare_dataframe(dataframe, has_user=True, has_fcreate=False, has_fmod=False):
    if has_user:
        dataframe['UsuarioModificacion'] = "@usuariomodificacion"
    if has_fcreate:
        dataframe['FechaCreacion'] = "@fechacreacion"
    if has_fmod:
        dataframe['FechaModificacion'] = "@fechamodificacion"
    return dataframe        

def prepare_sqlinput_rows(dataframe):
    
    rows = []
    for index, row in dataframe.iterrows():
        rows.append(row)

    dfvalues = ""

    for row in rows:

        temp_dict = row.to_dict()
        
        row_values = '('

        for k in temp_dict.keys():

            value = temp_dict[k]
            
            if type(value) == str:

                if value[0] == '@':
                    row_values = row_values + value + ", "
                else:
                    value = value.replace("'", "")
                    row_values = row_values + "'" + value + "', "
            
            elif type(value) == int:
                row_values = row_values + str(value) + ", "
            
            elif type(value) == type(np.NaN):
                row_values = row_values + str(value) + ", "
        row_values = "    " + row_values[:-2] + '),\n'
        
        dfvalues = dfvalues + row_values

    dfvalues = dfvalues[:-2] + ';'

    return dfvalues

def GET_SQL_INPUT(dataframe, table_name, use_database=True, database_name='Dbo', use_schema=True, schema_name='Dbo'):
    
    n = dataframe.shape[0]

    insert_cols = "("
    for c in dataframe.columns.tolist():
        insert_cols += c + ", "
    insert_cols = insert_cols[:-2] + ")"

    insert_block = "INSERT INTO "

    if use_database:
        insert_block += "[" + database_name + "]."
    if use_schema:
        insert_block += "[" + schema_name + "]."

    insert_block += "[" + table_name + "] " + insert_cols + "\nVALUES\n"


    if n <= 1000:

        dfvalues = prepare_sqlinput_rows(dataframe)
        output = insert_block + dfvalues

    else:

        s = int(np.floor(n / 1000))
        
        cdfs = []
        for i in range(s+1):
            cdfs.append(dataframe.iloc[i*1000:(i+1)*1000,:])

        output = ""
        for df in cdfs:
            dfvalues = prepare_sqlinput_rows(df)
            output = output + insert_block + dfvalues + '\n\n'

        output = output[:-2]
    
    return output

In [6]:
# sheets = [0]
# folder = 'input/DesarrolloClasificacionPrestacion/'

# files = [f for f in sorted(os.listdir(folder)) if not f.startswith('~$')]
# files = [f for f in files if 'clasificaciones' in f]
# file = folder + files[-1]

# dfs = {}
# for sh in sheets:
#     dfs[sh] = pd.read_excel(file, sheet_name=sh)
    
#     dfs[sh].FechaPrestaciones = [x.strftime('%Y-%m-%dT%H:%M:%S') for x in dfs[sh].FechaPrestaciones]

#     dfs[sh].loc[~dfs[sh].GlosaCorregida.isnull(), 'GlosaCorregida'] = dfs[sh].loc[~dfs[sh].GlosaCorregida.isnull(), 'GlosaCorregida'].str.capitalize()
#     dfs[sh].loc[~dfs[sh].GlosaProcesoVC.isnull(), 'GlosaProcesoVC'] = dfs[sh].loc[~dfs[sh].GlosaProcesoVC.isnull(), 'GlosaProcesoVC'].str.capitalize()
#     dfs[sh].loc[~dfs[sh].GlosaImed.isnull(), 'GlosaImed'] = dfs[sh].loc[~dfs[sh].GlosaImed.isnull(), 'GlosaImed'].str.capitalize()

# for sh in sheets:
#     dfs[sh] = prepare_dataframe(dfs[sh], has_user=hu, has_fcreate=hcd, has_fmod=hmd)



# print('Creando scripts de SQL ...')
# sqls = {}

# for sh in sheets:
#     print('     ...', 'Hoja1')
#     sqls[sh] = GET_SQL_INPUT(dfs[sh], table_name='DesarrolloClasificacionPrestacion', database_name="DesarrolloBI", schema_name="Prestaciones")
#     sqls[sh] = sqls[sh].replace('nan0', 'NULL')
# print('     ... done!')
# sql = ""

# if hu:
#     sql = "DECLARE @usuariomodificacion VARCHAR(25) = '" + user + "'" + "\n" 

# if hcd:
#     sql = sql + "DECLARE @fechacreacion DATETIME = GETDATE()" + "\n"

# if hmd:
#     sql = sql + "DECLARE @fechamodificacion DATETIME = GETDATE()" + "\n\n" 

# for sh in sheets:
#     sql = sql + sqls[sh] + '\n\n'

# today = date.today()
# today = today.strftime('%Y%m%d')

# now = datetime.now()
# now = now.strftime('%H%M')

# filename = 'output/create_upload_script/'+ script_name + '_' + today + '_' + now + '.sql'

# with open(filename, 'w') as f:
#     f.write(sql)        

In [45]:
#preparamos dataframe para hacer el sql
clasificados3 = prepare_dataframe(clasificados3,has_user=True,has_fcreate=True,has_fmod=True)
sqls = GET_SQL_INPUT(clasificados3,table_name='DesarrolloClasificacionPrestacion', database_name="DesarrolloBI", schema_name="Prestaciones")
sql = "  DECLARE @usuariomodificacion VARCHAR(25) = '" + 'AndreEscobar' + "'" + "\n" + "DECLARE @fechacreacion DATETIME = GETDATE()" + "\n" + "DECLARE @fechamodificacion DATETIME = GETDATE()" + "\n\n" + sqls + '\n\n'
sql = sql.replace("nan", "NULL")
clasificados3

today = date.today()
today = today.strftime('%Y%m%d')

now = datetime.now()
now = now.strftime('%H%M')

filename = 'output/create_upload_script/'+ 'UploadDesarrolloClasificacionPrestacion' + '_' + today + '_' + now + '.sql'

with open(filename, 'w') as f:
    f.write(sql)        




In [47]:
clasificados3.columns

Index(['CodigoVC', 'GlosaCorregida', 'GlosaProcesoVC', 'GlosaImed',
       'ClasificacionGrupo', 'ClasificacionSubgrupo', 'ClasificacionApertura',
       'FechaPrestacion', 'Vigencia', 'UsuarioModificacion', 'FechaCreacion',
       'FechaModificacion'],
      dtype='object')